In [2]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import gensim
import time, math
import numpy as np
import torch
import torch.nn as nn
import io
from torch.autograd import Variable
from bert_serving.client import BertClient

newsData = pd.read_csv("stockNews/Combined_News_DJIA.csv")
pd.set_option('display.max_colwidth', 2000)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wuhn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Word process

In [3]:
newsTrain = newsData[newsData['Date']< '2015-01-01']
newsTest = newsData[newsData['Date']>'2014-12-31']
labelTrain = newsTrain['Label'].values
labelTest = newsTest['Label'].values
newsTrain.head(2)
#print(newsTrain)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as countries move to brink of war""",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","b""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"b""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","b""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""",...,"b'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",b'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"b""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",b'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report',b'Caucasus in crisis: Georgia invades South Ossetia',"b'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",b'Visitors Suffering from Mental Illnesses Banned from Olympics',"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,"b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'",b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli training, we're fending off Russia """,b'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired',"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?',b'Russia angered by Israeli military sale to Georgia',b'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people',...,b'Israel and the US behind the Georgian aggression?',"b'""Do not believe TV, neither Russian nor Georgian. There are much more victims""'",b'Riots are still going on in Montreal (Canada) because police murdered a boy on Saturday.',b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Torture',b' Russia has just beaten the United States over the head with Peak Oil',b'Perhaps *the* question about the Georgia - Russia conflict ',b'Russia is so much better at war',"b""So this is what it's come to: trading sex for food."""


In [4]:
def news_to_words( news ):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", news) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [5]:
trainHeadLines = []
for row in range(0,len(newsTrain.index)):
#for row in range(0,2):
    #pre-process words
    tempList = []
    for col in range(2,27):
        combinedNews = " ".join(str(x).replace("b\"","").replace("b\'","") for x in newsTrain.iloc[row,col:col+1])
        #processedNews = news_to_words(combinedNews)
        tempList.append(combinedNews)
        if '' in tempList:
            print("empty",row,col)
    trainHeadLines.append(tempList)
# print(trainheadlines)
#print(len(newsTrain.index))

testHeadLines = []
for row in range(0,len(newsTest.index)):
#for row in range(0,2):
    #pre-process words
    tempList = []
    for col in range(2,27):
        combinedNews = " ".join(str(x).replace("b\"","").replace("b\'","") for x in newsTest.iloc[row,col:col+1])
        #processedNews = news_to_words(combinedNews)
        tempList.append(combinedNews)
        if '' in tempList:
            print("empty",row,col)
    testHeadLines.append(tempList)
#print(len(testHeadLines))
print(trainHeadLines[0])
#print(len(newsTrain.index))

['Georgia \'downs two Russian warplanes\' as countries move to brink of war"', "BREAKING: Musharraf to be impeached.'", "Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)'", "Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'", 'Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing"', "150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.'", 'Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side"', 'The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it."', "Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]'", "Did the U.S. Prep Georgia for War with Russia?'", "

# Create tensors

In [8]:
# Turn words list into list of longs
def word_tensor(sentenceList):
    varList = []
    bc = BertClient(check_length=False)
    #print(len(sentenceList))
    for c in range(len(sentenceList)):
        tempTensor = torch.from_numpy(bc.encode([sentenceList[c]]))
        varList.append(Variable(tempTensor))
    return varList

In [9]:
def label_tensor(label, length):
    tensor = torch.ones(length).long()
    tensor = tensor * label
    return Variable(tensor)

# Convert dataset to pytorch format

In [10]:
def convert_to_pytorch_format(X, y):
    M = len(X)
    X_processed = []
    y_processed = []

    
    for i in range(M):
        sequenceLength = 768
        #print(sequenceLength)
        inp = word_tensor(X[i])
        target = label_tensor(y[i], sequenceLength)
        X_processed.append(inp)
        y_processed.append(target)
        if i % 10 == 0:
            print("converted",i)
    return X_processed, y_processed
(X_train, y_train) = convert_to_pytorch_format(trainHeadLines, labelTrain)
(X_test, y_test) = convert_to_pytorch_format(testHeadLines, labelTest)

converted 0
converted 10
converted 20
converted 30
converted 40
converted 50
converted 60
converted 70
converted 80
converted 90
converted 100
converted 110
converted 120
converted 130
converted 140
converted 150
converted 160
converted 170
converted 180
converted 190
converted 200
converted 210
converted 220
converted 230
converted 240
converted 250
converted 260
converted 270
converted 280
converted 290
converted 300
converted 310
converted 320
converted 330
converted 340
converted 350
converted 360
converted 370
converted 380
converted 390
converted 400
converted 410
converted 420
converted 430
converted 440
converted 450
converted 460
converted 470
converted 480
converted 490
converted 500
converted 510
converted 520
converted 530
converted 540
converted 550
converted 560
converted 570
converted 580
converted 590
converted 600
converted 610
converted 620
converted 630
converted 640
converted 650
converted 660
converted 670
converted 680
converted 690
converted 700
converted 710
con

# File Output

In [39]:
print(X_train[0][0].size)
print(len(X_train[0]))
tempWriteList = []
for i in range(len(X_train)):
    for j in range(len(X_train[0])):
        tempWriteList.append(X_train[i][j])
print(len(tempWriteList))
for i in range(len(tempWriteList)):
    tempWriteList[i] = tempWriteList[i][0]
np.savetxt('xtrain.txt',tempWriteList)

768
25
40275


In [47]:
for i in range(len(X_train)):
    for j in range(len(X_train[0])):
        X_train[i][j] = torch.from_numpy(X_train[i][j])

<class 'torch.Tensor'>


In [50]:
tempWriteList = []
for i in range(len(X_test)):
    for j in range(len(X_test[0])):
        #X_test[i][j] = X_test[i][j].numpy()
        tempWriteList.append(X_test[i][j])
for i in range(len(tempWriteList)):
    tempWriteList[i] = tempWriteList[i][0]
np.savetxt('xtest.txt',tempWriteList)

In [51]:
for i in range(len(X_test)):
    for j in range(len(X_test[0])):
        X_test[i][j] = torch.from_numpy(X_test[i][j])

<class 'torch.Tensor'>


# RNN

In [57]:
class RNN(nn.Module):
    # define model architecture
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        
        #(self.embedding, num_embeddings, embedding_dim) = create_emb_layer(weights_matrix, non_trainable=False)
        #self.embedding = create_emb_layer()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        # encoder, GRU, decode
        # self.embeddings = nn.Embedding.from_pretrained(weights)
        # self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.model = nn.Linear(hidden_size, output_size)
    
    # forward prop definition  FOR A SINGLE ELEMENT IN THE SEQUENCE
    def forward(self, input, hidden):
        #input = self.embedding(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.model(output.view(1, -1))
        return output, hidden

    # Hidden state initialization for a single sequence
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
    
    # forward propagate and return last layer softmax output
    def output_softmax(self, line_tensor):
        hidden = self.init_hidden()
        # loop thru sequences, make continuous prediction
        for i in range(len(line_tensor)):
            output, hidden = self(line_tensor[i], hidden)
        return output
    
    
    # predict function
    def predict_dataset(self, dataset):
        predictions = []
        for i in range(len(dataset)):
            output = self.output_softmax(dataset[i])
            predictedLabel = output.argmax()
            predictions.append(predictedLabel)
        return np.array(predictions)
    
    def accuracy(self, dataset, labels):
        m = len(labels)
        predictions = self.predict_dataset(dataset)
        correctCount = np.sum(predictions == labels)
        accuracy = correctCount/m
        return accuracy

In [58]:
def train(inp, target):
    string_len = len(inp)
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for c in range(string_len):
        output, hidden = model(inp[c], hidden)
        loss += criterion(output, target[c].unsqueeze(0))

    loss.backward()
    model_optimizer.step()
    
    return loss.data.item() / string_len

In [59]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [63]:
testSampleNo = 0
trainSampleNo = 0

n_epochs = 100
print_every = 100
evaluate_every = 50
hidden_size = 100
vector_size = 768
output_size = 2
n_layers = 1
lr = 0.008
trainM = len(X_train)
testM = len(X_test)

model = RNN(vector_size, hidden_size, output_size, n_layers)
model_optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    for i in range(trainM):
        loss = train(X_train[i], y_train[i])
        loss_avg += loss
        if i % evaluate_every == 0:
            print(i , " step, accuracy = ", model.accuracy(X_test, labelTest))
    print(epoch , " epoch, accuracy = ", model.accuracy(X_test, labelTest))

0  step, accuracy =  0.49206349206349204
50  step, accuracy =  0.49206349206349204
100  step, accuracy =  0.49206349206349204
150  step, accuracy =  0.49206349206349204
200  step, accuracy =  0.49206349206349204
250  step, accuracy =  0.5079365079365079
300  step, accuracy =  0.49206349206349204
350  step, accuracy =  0.5079365079365079
400  step, accuracy =  0.5079365079365079
450  step, accuracy =  0.49206349206349204
500  step, accuracy =  0.5079365079365079
550  step, accuracy =  0.5079365079365079
600  step, accuracy =  0.5079365079365079
650  step, accuracy =  0.5079365079365079
700  step, accuracy =  0.5079365079365079
750  step, accuracy =  0.49206349206349204
800  step, accuracy =  0.5079365079365079
850  step, accuracy =  0.49206349206349204
900  step, accuracy =  0.5079365079365079
950  step, accuracy =  0.49206349206349204
1000  step, accuracy =  0.49206349206349204
1050  step, accuracy =  0.5079365079365079
1100  step, accuracy =  0.5079365079365079
1150  step, accuracy = 

1150  step, accuracy =  0.5105820105820106
1200  step, accuracy =  0.5079365079365079
1250  step, accuracy =  0.5079365079365079
1300  step, accuracy =  0.49206349206349204
1350  step, accuracy =  0.49206349206349204
1400  step, accuracy =  0.5079365079365079
1450  step, accuracy =  0.5079365079365079
1500  step, accuracy =  0.5079365079365079
1550  step, accuracy =  0.49206349206349204
1600  step, accuracy =  0.5079365079365079
6  epoch, accuracy =  0.5105820105820106
0  step, accuracy =  0.5105820105820106
50  step, accuracy =  0.49206349206349204
100  step, accuracy =  0.5185185185185185
150  step, accuracy =  0.49206349206349204
200  step, accuracy =  0.49206349206349204
250  step, accuracy =  0.5079365079365079
300  step, accuracy =  0.49206349206349204
350  step, accuracy =  0.5079365079365079
400  step, accuracy =  0.5079365079365079
450  step, accuracy =  0.49206349206349204
500  step, accuracy =  0.5079365079365079
550  step, accuracy =  0.5079365079365079
600  step, accuracy 